Dog vs. Cats example

In [20]:
from scipy import io
import numpy as np
import os


# dogs vs. cats
dogdata_w_mat = io.loadmat(os.path.join('..','DATA','dogData_w.mat'))
catdata_w_mat = io.loadmat(os.path.join('..','DATA','catData_w.mat'))

dog_wave = dogdata_w_mat['dog_wave']
cat_wave = catdata_w_mat['cat_wave']

CD = np.concatenate((dog_wave,cat_wave),axis=1)

train = np.concatenate((dog_wave[:,:60],cat_wave[:,:60]),axis=1)
test = np.concatenate((dog_wave[:,60:80],cat_wave[:,60:80]),axis=1)
label = np.repeat(np.array([1,-1]),60)
label_test = np.repeat(np.array([1,-1]),20)


In [21]:
# print progress, possibly faster than progress bar
def print_progress(epoch, batch_id, accuracy, loss):
    progress = '='* int((10. * batch_id / len(train_data)))
    progress += '>'
    if batch_id == 1:
        print('Train Epoche {}: {}% [{}]\t accuracy: {:.6f}, loss: {:.6f}'.format(
            epoch, 100. * batch_id / len(train_data),progress, accuracy, loss.item()), end='')
    else:
        print('\rTrain Epoche {}: {}% [{}]\t accuracy: {:.6f}, loss: {:.6f}'.format(
            epoch, 100. * batch_id / len(train_data),progress, accuracy, loss.item()), end='', flush = True)

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms

# shuffle data
idx_col = np.random.permutation(train.shape[1])
train_data_list = np.ones((train.shape[0], idx_col.shape[0]))
label_data_list = np.ones((2, idx_col.shape[0]))
train_data_list[:,0:] = train[:,idx_col[:]]
label_data_list[0:] = label[idx_col[:]]
for j in range(0,label_data_list.shape[1]):         # dog: first row[0], cat: second row[1]
    if label_data_list[0,j] == 1:
        label_data_list[1,j] = 0                    # 1=dog, -1=cat
    elif label_data_list[0,j] == -1:
        label_data_list[0,j] = 0
        label_data_list[1,j] = 1
        
train_data = []
train_data.append((torch.from_numpy(np.transpose(train_data_list)), torch.from_numpy(np.transpose(label_data_list))))

idx_col = np.random.permutation(test.shape[1])
test_data_list = np.ones((test.shape[0], idx_col.shape[0]))
test_data_list[:,0:] = test[:,idx_col[:]]
label_test_list = np.ones((2, idx_col.shape[0]))
label_test_list[0:] = label_test[idx_col[:]]
for j in range(0,label_test_list.shape[1]):
    if label_test_list[0,j] == 1:
        label_test_list[1,j] = 0
    elif label_test_list[0,j] == -1:
        label_test_list[0,j] = 0
        label_test_list[1,j] = 1

test_data = []
test_data.append((torch.from_numpy(test_data_list), torch.from_numpy(label_test_list)))

#print(train_data)


AttributeError: 'numpy.ndarray' object has no attribute 'expand_dims'

In [18]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.conv1 = nn.Conv1d(1,20,kernel_size=5)
        self.conv2 = nn.Conv1d(20,50,kernel_size=5)
        self.conv_dropout = nn.Dropout2d()            # damit das Netz nichts auswendig lernt
        #self.conv3 = nn.Conv1d(50, 40, kernel_size=3)
        #self.fc2 = nn.Linear(60, 2)
    
    def forward(self,x):
        x = self.conv1(x)
        x = F.max_pool1d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv_dropout(x)
        x = F.max_pool1d(x, 2)
        x = F.relu(x)
        #x = x.view(-1,320)                             # daten umformen
        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)
        print('test:',x.shape)
        
        return F.log_softmax(x)
        

# model
model = net()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.8)   # momentum [0,1]

# training
def train(epoch):
    model.train()
    batch_id = 1
    for data, target in train_data:
        optimizer.zero_grad()
        
        print(data.shape)
        # forward pass
        out = model(data)
        
        #calculate loss
        loss = F.nll_loss(out, target)          # nn.CrossEntropyLoss(), F.nll_loss is better
        loss.backward()
        optimizer.step()
        # display training progress
        print_progress()
        #print('Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #epoch,batch_id*len(data), len(train_data.dataset), 100. * batch_id / len(train_data), loss.item()))
        accuracy = 100. * batch_id / len(train_data)
        torch.save(net, 'data_figures_nn/MyNet_DogCat.pt')
        batch_id += 1

def test():
    model.eval()
        
for epoch in range(1,2):
        train(epoch)


torch.Size([1])


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [20, 1, 5], but got 1-dimensional input of size [1] instead

Dog vs. Cats with a tensorflow model (original from the book)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import io
import torch 
import os
from sklearn import linear_model


rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 12]

In [ ]:
# dogs vs. cats
dogdata_w_mat = io.loadmat(os.path.join('..','DATA','dogData_w.mat'))
catdata_w_mat = io.loadmat(os.path.join('..','DATA','catData_w.mat'))

dog_wave = dogdata_w_mat['dog_wave']
cat_wave = catdata_w_mat['cat_wave']

CD = np.concatenate((dog_wave,cat_wave),axis=1)

train = np.concatenate((dog_wave[:,:60],cat_wave[:,:60]),axis=1)
test = np.concatenate((dog_wave[:,60:80],cat_wave[:,60:80]),axis=1)
label = np.repeat(np.array([1,-1]),60)

A = label @ np.linalg.pinv(train)
test_labels = np.sign(A@test)

In [ ]:
lasso = linear_model.Lasso().fit(train.T,label)
A_lasso = lasso.coef_
test_labels_lasso = np.sign(A_lasso@test)

In [ ]:
fig,axs = plt.subplots(4,1)
axs[0].bar(range(len(test_labels)),test_labels)
axs[1].bar(range(len(A)),A)
axs[2].bar(range(len(test_labels_lasso)),test_labels_lasso)
axs[3].bar(range(len(A_lasso)),A_lasso)


plt.show()

In [ ]:
fig,axs = plt.subplots(2,2)
axs = axs.reshape(-1)
A2 = np.flipud(np.reshape(A,(32,32)))
A2_lasso = np.flipud(np.reshape(A_lasso,(32,32)))
axs[0].pcolor(np.rot90(A2),cmap='gray')
axs[1].pcolor(np.rot90(A2_lasso),cmap='gray')


plt.show()